# Multi Objective Evolutionary Algorithms (Part 2)

In this chapter we continue with multi-objective search algorithms, and consider some alternatives to NSGA-II. First we need to import the usual stuff, set up our wrapper class and plotting infrastructure.

## Setting the Scene

In [ ]:
import sys
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from math import sqrt

As an example problem instance, we consider the Next Release Problem (NRP) again.

In [ ]:
profit_map = {}        # customer id => weight
requirements_map = {}  # customer id => [ requirement_id * ]
cost_map = {}          # requirement id => cost
dependency_map = {}    # requirement id => [ requirement_id * ]
num_requirements = 140

We'll need our parser for the standard format again.

In [ ]:
def parse_nrp(filename):

    with open(filename) as fp:
        levels   = int(fp.readline())
        req_id = 1
        for level in range(levels):
            num_reqs = int(fp.readline())
            for cost in fp.readline().split():
                cost_map[req_id] = int(cost)
                req_id += 1
        total_deps = int(fp.readline())
        for num_dep in range(total_deps):
            r1, r2 = fp.readline().split()
            r1_id = int(r1)
            if r1_id in dependency_map:
                dependency_map[int(r1)].append(int(r2))
            else:
                dependency_map[int(r1)] = [int(r2)]

        total_customers = int(fp.readline())
        for num_cust in range(total_customers):
            customer = fp.readline().split()
            profit_map[num_cust + 1] = int(customer[0])
            num_reqs = int(customer[1])
            requirements = []
            for num_req in range(num_reqs):
                requirements.append(int(customer[2+num_req]))
            requirements_map[num_cust + 1] = requirements

        num_requirements = len(cost_map)

As running example, we will use the small instance `nrp1.txt`:

In [ ]:
parse_nrp("data/nrp/nrp1.txt")

Our first objective is to maximise the profit:

In [ ]:
# Profit is the sum of the weights of customers whose requirements are satisfied
def function1(solution):
    fitness = 0

    requirements = set([x+1 for x in range(len(solution)) if solution[x] == 1])
    for customer_id in profit_map.keys():
        reqs = set(requirements_map[customer_id])
        if reqs.issubset(requirements):
            fitness += profit_map[customer_id]

    return fitness

The second objective is to minimise the costs:

In [ ]:
# Cost is the sum of costs of the implemented requirements
def function2(solution):
    cost = 0

    for i in range(num_requirements):
        if solution[i]:
            cost += cost_map[i+1]

    return cost

In [ ]:
total_costs = sum([cost_map[i+1] for i in range(num_requirements)])

In [ ]:
total_profit = sum(profit_map.values())

To avoid redundantly calculating fitness values, we will use the wrapper class for the list to cache values in attributes.

In [ ]:
class L(list):
    """
    A subclass of list that can accept additional attributes.
    Should be able to be used just like a regular list.
    """
    def __new__(self, *args, **kwargs):
        return super(L, self).__new__(self, args, kwargs)

    def __init__(self, *args, **kwargs):
        if len(args) == 1 and hasattr(args[0], '__iter__'):
            list.__init__(self, args[0])
        else:
            list.__init__(self, args)
        self.__dict__.update(kwargs)

    def __call__(self, **kwargs):
        self.__dict__.update(kwargs)
        return self

We can now store the fitness values of individuals as attributes of the objects

In [ ]:
def evaluate(individual):
    individual.fitness1 = function1(individual)
    individual.fitness2 = function2(individual)

Individuals of NRP solutions are instances of `L` rather than lists, and we can define our usual search operators for bitvector representation:

In [ ]:
def get_random_individual():
    individual = L(random.choice([0,1]) for _ in range(num_requirements))
    evaluate(individual)
    return individual

In [ ]:
def mutate(solution):
    P_mutate = 1/len(solution)
    mutated = L(solution[:])
    for position in range(len(solution)):
        if random.random() < P_mutate:
            mutated[position] = 1 - mutated[position]
    evaluate(mutated)
    return mutated

In [ ]:
def crossover(parent1, parent2):
    pos = random.randint(0, len(parent1))
    offspring1 = L(parent1[:pos] + parent2[pos:])
    offspring2 = L(parent2[:pos] + parent1[pos:])
    return (offspring1, offspring2)

In addition to the representation, we also want to keep using animations to study how the algorithms explore the search space:

In [ ]:
ims = []  # global variable to store images of the animation
    
def initialise_plot():
    global ims
    global fig
    global ax
    
    ims = []

    %matplotlib agg
    fig, ax = plt.subplots()
    plt.xlabel('Profit', fontsize=15)
    plt.ylabel('Cost', fontsize=15)
    ims = []
    %matplotlib inline

For each iteration of the algorithm, we will update this animation with a snapshot of the current population and their fitness values:

In [ ]:
def plot(population):
    function1_values = [x.fitness1 for x in population]
    function2_values = [x.fitness2 for x in population]
    
    ims.append((ax.scatter(function1_values, function2_values, color="blue"),))

## Baseline 1: Random Search

When studying different search algorithms, random search always servers as a sanity check to compare against. To generalise random search to multi-objective random search, we'll need to define the dominance relation again.

In [ ]:
def dominates(solution1, solution2):
    """
     A solution x(1) is said to dominate the other solution x(2) if both condition 1 and 2 below are true:

     Condition 1: x(1) is no worse than x(2) for all objectives
     Condition 2: x(1) is strictly better than x(2) in at least one objective
     
     We are maximising fitness 1, but minimising fitness 2
    """
    if solution1.fitness1 < solution2.fitness1 or solution1.fitness2 > solution2.fitness2:
        return False

    if solution1.fitness1 > solution2.fitness1 or solution1.fitness2 < solution2.fitness2:
        return True

    return False

An operation we will frequently need in this chapter, and also for random search, is to extract the non-dominated solutions for a given collection of solutions.

In [ ]:
def get_nondominated(population):
    nondominated = []

    for x in population:
        dominated = False
        for y in population:
            if dominates(y, x):
                dominated = True
                break
        if not dominated:
            nondominated.append(x)

    return nondominated

Given these ingredients, we can generalise random search to multi-objective random search by repeatedly sampling random individuals, and keeping those that are not dominated.

In [ ]:
max_gen = 100
population_size = 20

In [ ]:
def random_moo():
    initialise_plot()
    result = []
    for iteration in range(max_gen * population_size):
        candidate = get_random_individual()
        result = get_nondominated(result + [candidate])
        if iteration % population_size == 0:
            plot(result)
    return result

Similar to the last chapter, we can look at the evolution for each algorithm by plotting the solutions found for each iteration (or in this case, after each `population_size` individuals have been evaluated, to speed up the animation).

In [ ]:
result = random_moo()

In [ ]:
im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000, blit=True)
HTML(im_ani.to_jshtml())

To systematically compare sarch algorithms, we need to run experiments with multiple repetitions again. For this we use our usual helper function.

In [ ]:
from IPython.utils import io

def run_times(algorithm, repetitions):
    global ims
    result = []
    for i in range(repetitions):
        ims = []
        with io.capture_output() as captured: 
            front = algorithm()
        result.append(front)
    return result

We also need a metric to compare algorithms with. In the last chapter we considered several different metrics, in particular the hypervolume.

In [ ]:
def hypervolume(front, r):
    front.sort(key=lambda i: i.fitness1)

    hv = (abs(r[0] - front[0].fitness1) / total_profit) * (abs(r[1] - front[0].fitness2) / total_costs)
    
    for i in range(1, len(front)):
        hv += (abs(front[i-1].fitness1 - front[i].fitness1) / total_profit) * (abs(r[1] - front[i].fitness2) / total_costs)
    
    return hv

We will keep all experiment results in a dictionary `results` such that we can produce comparative boxplots throughout this chapter.

In [ ]:
fronts_random = run_times(random_moo, 10)
results = {}
results["Random"] = [hypervolume(front, (0,total_costs)) for front in fronts_random]

## Baseline 2: NSGA-II

In the last chapter we introduced NSGA-II as our first multi-objective search algorithm, and indeed it is one of the most popular multi-objective algorithms and has been shown to be effective and efficient for many different problems. Hence we definitely need to include it in our comparison in this chapter. In the following, the individual bits and pieces of NSGA-II from the previous chapter are re-introduced.

In [ ]:
def fast_non_dominated_sort(solutions):
    front = [[]]

    S = [[] for _ in range(len(solutions))]
    n = [0 for _ in range(len(solutions))]

    for p in range(len(solutions)):
        S[p] = []
        n[p] = 0
        for q in range(len(solutions)):
            if dominates(solutions[p], solutions[q]):
                S[p].append(q)
            elif dominates(solutions[q], solutions[p]):
                n[p] = n[p] + 1

        if n[p] == 0:
            front[0].append(p)
            solutions[p].rank = 0

    i = 0
    while front[i]:
        Q = []
        for p in front[i]:
            for q in S[p]:
                n[q] = n[q] - 1
                if n[q] == 0:
                    Q.append(q)
                    solutions[q].rank = i + 1
        i = i + 1
        front.append(Q)

    del front[-1]
    return front

In [ ]:
def calculate_crowding_distance_and_sort(front):

    data = [(x, front[x].fitness1, front[x].fitness2) for x in range(len(front))]
    sorted1 = [(x, y) for (x, y, z) in sorted(data, key=lambda tup: tup[1])]
    sorted2 = [(x, z) for (x, y, z) in sorted(data, key=lambda tup: tup[2])]

    distance = [0 for _ in range(0,len(front))]
    range_fitness1 = max(x.fitness1 for x in front) - min(x.fitness1 for x in front)
    range_fitness2 = max(x.fitness2 for x in front) - min(x.fitness2 for x in front)

    distance[sorted1[0][0]] = sys.maxsize
    distance[sorted1[-1][0]] = sys.maxsize

    distance[sorted2[0][0]] = sys.maxsize
    distance[sorted2[-1][0]] = sys.maxsize

    for k in range(1,len(front)-1):
        index = sorted1[k][0]
        distance[index] = distance[index] + (sorted1[k+1][1] - sorted1[k-1][1]) / range_fitness1

    for k in range(1,len(front)-1):
        index = sorted2[k][0]
        distance[index] = distance[index] + (sorted2[k+1][1] - sorted2[k-1][1]) / range_fitness2

    for k in range(0, len(front)):
        front[k].distance = distance[k]

    front.sort(key = lambda i: i.distance, reverse=True)

In [ ]:
def binary_rank_tournament(population):
    individual1 = random.choice(population)
    individual2 = random.choice(population)

    if individual1.rank < individual2.rank:
        return individual1
    elif individual1.rank > individual2.rank:
        return individual2
    else:
        return max([individual1, individual2], key = lambda i: i.distance)

In [ ]:
def generate_offspring(population):
    offspring_population = []
    while len(offspring_population) < len(population):
        parent1 = binary_rank_tournament(population)
        parent2 = binary_rank_tournament(population)
        if random.random() < P_xover:
            offspring1, offspring2 = crossover(parent1, parent2)
        else:
            offspring1, offspring2 = parent1, parent2
            
        offspring1 = mutate(offspring1)
        offspring2 = mutate(offspring2)

        offspring_population.append(offspring1)
        offspring_population.append(offspring2)

    return offspring_population

In [ ]:
def get_initial_population(population_size):
    population = [get_random_individual() for _ in range(population_size)]
    fronts = fast_non_dominated_sort(population)
    
    for front_indices in fronts:
        front = [population[index] for index in front_indices]
        calculate_crowding_distance_and_sort(front)

    return population

In [ ]:
def nsgaii():
    initialise_plot()
    population = get_initial_population(population_size)
    offspring_population = generate_offspring(population)

    for iteration in range(max_gen):
        combined = population + offspring_population
        #plot(combined)
        fronts = fast_non_dominated_sort(combined)
        population = []

        for front_indices in fronts:
            front = [combined[index] for index in front_indices]
            calculate_crowding_distance_and_sort(front)

            for i in front:
                population.append(i)
                if len(population) == population_size:
                    break
            if len(population) == population_size:
                break

        plot(population)
        offspring_population = generate_offspring(population)

    non_dominated_sorted_solution = fast_non_dominated_sort(population)
    result = [population[x] for x in non_dominated_sorted_solution[0]]
    plot(result)

    return result

First let's consider the evolution of an individual run again.

In [ ]:
P_xover = 0.7
result = nsgaii()

In [ ]:
im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000, blit=True)
HTML(im_ani.to_jshtml())

We also need to collect some more data for comparison.

In [ ]:
fronts_nsgaii = run_times(nsgaii, 10)
results["NSGA-II"] = [hypervolume(front, (0,total_costs)) for front in fronts_nsgaii]

Our first sanity check is whether NSGA-II is indeed better than random search.

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(results.values())
ax.set_xticklabels(results.keys())

## PAES

As first alternative example of a multi-objective search algorithm, we consider the Pareto Archived Evolution Strategy (PAES):

J. Knowles, D. Corne. “The pareto archived evolution strategy: A new baseline algorithm for pareto multiobjective optimisation.” In Congress on Evolutionary Computation (CEC99) (Vol. 1, pp. 98-105). 1999

PAES is intended to be the simplest possible non-trivial algorithm capable of generating diverse solutions in the Pareto optimal set. It essentiall is a (1+1)ES adapted for multi-objective search, and is intended as baseline approach for evaluation of other algorithms.

Like a (1+1)ES the algorithm has a population of size 1 and produces 1 offspring. In addition, there is an archive which stores non-dominated solutions (subject to diversity criteria). At the end of the run, the archive is the final result.

1. Generate random current solution
2. Evaluate and add to archive
3. While not done:
4. Create candidate by mutating current solution
5. If candidate is dominated by current, reject it
6. Else if current is dominated by the candidate, accept candidate and add it to the archive
7. Else Compare candidate solutions with archive members, update archive and current

The archive stores only non-dominated solutions, and has a maximum size. The archive is also used in order to decide whether or not to accept an offspring:
- Candidates which dominate the archive are always accepted and archived.
- Candidates which are dominated by the archive are always rejected.
- Non-dominated are accepted/archived based on how many similar individuals already exist

When an individual is added to the archive, we need to remove all individuals from the archive that are dominated by this new member. However, when the maximum archive size is reached, we may have to decide which individuals to keep in the archive. In this case, PAES checks if the new candidate would increase the diversity in the archive. If so, it replaces the individual risiding in the most crowded part of the archive.

We start by defining some helper functions since PAES works with an archive. First, we add a function that tells us if a candidate individual is dominated by any of the individuals in the archive:

In [ ]:
def is_dominated(candidate, archive):
    for i in archive:
        if dominates(i, candidate):
            return True
    return False

We also need an update operation that removes all dominated individuals from an archive:

In [ ]:
def remove_dominated(candidate, archive):
    copy = [x for x in archive if not dominates(candidate, x)]
    archive.clear()
    archive.extend(copy)

Third,we need to select all solutions in the archive that are dominated by an individual, as we want to remove all of them when adding a new non-dominated solution:

In [ ]:
def get_dominated(candidate, archive):
    return [x for x in archive if dominates(candidate, x)]

Finally, we need an operation that updates the archive. As we are using lists but technically the archive should be an set for PAES, we'll have to add a check here (inefficient, but shorter than redefining previous operations to work on sets rather than lists):

In [ ]:
def add_to_archive(individual, archive):
    if not individual in archive:
        archive.append(individual)

When there is dominance, the choice of what to do is easy. The more tricky case is when the new individual is not dominated by the archive. As the archive has a maximum size, we need to decide which individuals to keep once we hit the maximum. In PAES, this is done using a grid from which we can infer which areas of the objective space are more crowded. 

In [ ]:
grid_size = 10

def get_grid(archive):
    grid = {}
    max_profit = max([i.fitness1 for i in archive])+1
    max_effort = max([i.fitness2 for i in archive])+1
    f1step = max_profit / grid_size
    f2step = max_effort / grid_size

    for x in range(grid_size):
        grid[x] = {}
        for y in range(grid_size):
            grid[x][y] = []

    for individual in archive:
        x = individual.fitness1 // f1step
        y = individual.fitness2 // f2step
        grid[x][y].append(individual)

    return grid

We want to add individuals to the grid if they help us preserve diversity. Thus, we check if we would add the individual to the most crowded grid cell. If we are, then we are not increasing diversity.

In [ ]:
def increases_diversity(candidate, grid, archive):
    max_crowd = 0
    target_crowd = 0

    # We have to redundantly calculate this to figure out the step size...
    max_profit = max([i.fitness1 for i in archive]) + 1
    max_cost = max([i.fitness2 for i in archive]) + 1
    f1step = max_profit / grid_size
    f2step = max_cost / grid_size

    target_x = candidate.fitness1 // f1step
    target_y = candidate.fitness2 // f2step
    
    for x in range(grid_size):
        for y in range(grid_size):
            num = len(grid[x][y])
            if num > max_crowd:
                max_crowd = num
            if target_x == x and target_y == y:
                target_crowd = num

    return target_crowd < max_crowd

We can also compare individuals in terms of how crowded their grid cells are:

In [ ]:
def less_crowded_than(individual1, individual2, grid):
    individual1_crowd = 0
    individual2_crowd = 0

    for x in range(grid_size):
        for y in range(grid_size):
            if individual1 in grid[x][y]:
                individual1_crowd = len(grid[x][y])
            if individual2 in grid[x][y]:
                individual2_crowd = len(grid[x][y])

    return individual1_crowd < individual2_crowd

If we need to reduce the size of the archive, we randomly pick one individual from the most crowded grid cell:

In [ ]:
def remove_from_grid(grid, archive):
    pos_x, pos_y = 0, 0
    max_crowd = 0

    for x in range(grid_size):
        for y in range(grid_size):
            num = len(grid[x][y])
            if num > max_crowd:
                max_crowd = num
                pos_x, pos_y = x, y

    selected = random.choice(grid[pos_x][pos_y])
    archive.remove(selected)

The PAES algorithm itself mainly consists of the logic to decide what to do in case of non-domination:

In [ ]:
def paes():
    initialise_plot()
    archive = []
    current = get_random_individual()
    add_to_archive(current, archive)

    for step in range(population_size * max_gen):
        candidate = mutate(current)
        while candidate == current or candidate in archive:
            candidate = mutate(current)
        if step % population_size == 0:
            plot(archive + [candidate])

        if not dominates(current, candidate) and not is_dominated(candidate, archive):
            dominated_archive = get_dominated(candidate, archive)
            if dominated_archive:
                # If the candidate dominates something in the archive
                # we keep it in the archive and make it the new current
                remove_dominated(candidate, archive)
                add_to_archive(candidate, archive)
                current = candidate
            else:
                if len(archive) == population_size:
                    # Maximum archive size reached
                    grid = get_grid(archive)
                    if increases_diversity(candidate, grid, archive):
                        remove_from_grid(grid, archive)
                        add_to_archive(candidate, archive)
                        grid = get_grid(archive)
                        if less_crowded_than(candidate, current, grid):
                            current = candidate
                else:
                    # Enough space in archive
                    add_to_archive(candidate, archive)
                    grid = get_grid(archive)
                    if less_crowded_than(candidate, current, grid):
                        current = candidate

    return get_nondominated(archive)

In [ ]:
result = paes()

In [ ]:
im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000, blit=True)
HTML(im_ani.to_jshtml())

As always, we add some data and compare.

In [ ]:
fronts_paes = run_times(paes, 10)
results["PAES"] = [hypervolume(front, (0,total_costs)) for front in fronts_paes]

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(results.values())
ax.set_xticklabels(results.keys())

## SPEA-2

The Strength Pareto Evolutionary Algorithm (SPEA) version 2 is often treated as the main competitor of NSGA-II. It was defined in the following paper:

E. Zitzler, M. Laumanns and L. Thiele, "SPEA2: Improving the strength pareto evolutionary algorithm for multiobjective optimization", in Evolutionary Methods for Design, Optimisation and Control with Application to Industrial Problems, 2002.

This algorithm assigns a fitness to an individual based on the strength of its dominators. This fitness value is then minimised to produce solutions that are not dominated.

Another central distinguishing feature is the use of an archive.

The overall workflow of the algorithm is as follows:
1. Create initial population $P_0$ and create an empty archive $\bar{P}_0$.
2. Evaluate fitness values of the individuals in $P_t$ and $\bar{P}_t$
3. Copy all non-dominated individuals in $P_t$ and $\bar{P}_t$ to $\bar{P}_{t+1}$. If $\bar{P}_{t+1}$ exceeds the maximum archive size then truncate; if it is smaller then fill up with dominated individuals from $P_t$ and $\bar{P}_t$.
4. Perform binary tournament selection with replacement on $\bar{P}_{t+1}$ to fill the mating pool
5. Apply recombination and mutation operators to the mating pool and set $P_{t+1}$ to the resulting population.
6. Repeat from step 2 until done

The solution is represented by the non-dominated individuals in $\bar{P}$ at the end.

Let's first look at how the fitness values are determined. The score of an individual is the number of solutions that it dominates:

$Score(i) = |j \; | \; j \in P_t \cup \bar{P}_t \wedge i 	\succ j   |$

Here, $i \succ j$ denotes that $i$ dominates $j$. (Note that Zitzler et al. use the operator $\succ$ the other way round as Deb et al.)

In [ ]:
def score(individual, combined):
    score = 0
    for other in combined:
        if dominates(individual, other):
            score += 1

    individual.score = score

The raw fitness value of an individual is calculated as the sum of its dominators' strengths:

In [ ]:
def raw_fitness(individual, combined):
    raw_fitness = 0
    for other in combined:
        if dominates(other, individual):
            raw_fitness += other.score

    individual.raw_fitness = raw_fitness

The fitness value considers not only the raw fitness, but also the density, such that individuals in less populated areas of the search space are preferred:

$Density(i) = \frac{1}{\sigma_i^k + 2}$

Here, $\sigma_i^k$ denotes the distance of $i$ to its $k$th neighbour, and $k = \sqrt{N + \bar{N}}$, where $N$ is the size of the population and $\bar{N}$ is the size of the archive.

The distance $\sigma$ is defined in terms of the objective values:

In [ ]:
def get_distance(individual, other):
    sum = 0

    sum += (individual.fitness1 - other.fitness1) ** 2
    sum += (individual.fitness2 - other.fitness2) ** 2

    return sqrt(sum)

Now we just need to calculate these distances, sort, and assign to the individuals:

In [ ]:
def density(individual, combined):
    k = int(sqrt(len(combined)))
    
    distances = []
    for j in range(len(combined)):
        other = combined[j]
        if individual == other:
            continue
        distances.append(1/(2.0 + get_distance(individual, other)))
    distances.sort()
    individual.distance = distances[k]        

To put it all together, the following function calculates the fitness (strength) of all individuals given a population and archive:

In [ ]:
def calculate_strength(population, archive):
    combined = population + archive

    for x in combined:
        score(x, combined)
    for x in combined:
        raw_fitness(x, combined)
    for x in combined:
        density(x, combined)
    
    for individual in combined:
        individual.fitness = individual.raw_fitness + individual.distance

The other unique aspect of this algorithm is the handling of the archive. The size of the archive is constant over time, which means that sometimes individuals need to be removed or added in order to adjust the size. 

The _truncate_ function removes individuals from the archive; at each step, the individual which has the minimum distance to another individual is chosen for removal until the archive is no longer too large. When multiple individuals have the same distance to their closest neighbour, they are compared against the next closest neighbour until a difference is found. For this, we need a custom comparison operator:

In [ ]:
def compare_distances(distances1, distances2):
    for i in range(len(distances1)):
        if distances1[i] != distances2[i]:
            return distances1[i] - distances2[i]
    return 0

The truncate function now needs to determine all distances and then sort the individuals in order to decide which ones to drop:

In [ ]:
from functools import cmp_to_key
def truncate(archive):
    # Remove individual with minimum distance
    num_remove = abs(population_size - len(archive))

    for i in range(len(archive)):
        individual = archive[i]
        distances = []
        for j in range(len(archive)):
            if i == j:
                continue
            other = archive[j]
            distances.append(get_distance(individual, other))
        distances.sort()
        individual.distances = distances
        
        # This ignores the case that multiple individuals have the same distance
        # in which case we need to look at the next distance

    def compare(x, y):
        return compare_distances(x.distances, y.distances)
    archive.sort(key=cmp_to_key(compare))
    for i in range(num_remove):
        del archive[0]

Since Python >= 3 only supports the use of keys to sort lists, this function uses some extra code to wrap out `compare_distances` function as a key function.

If the archive is too small, then we fill it with the best dominated individuals from the population. Thus, we sort the population by fitness (i.e., strength), and then pick from the sorted list.

In [ ]:
def pad_archive(next_archive, population, archive):
    num_missing = population_size - len(next_archive)

    # fill with dominated individuals in archive and population
    candidates = [i for i in population+archive if i.fitness >= 1]
    candidates.sort(key = lambda r : r.fitness)
    next_archive.extend(candidates[:num_missing])

The mating is the same we already know from other evolutionary algorithms. The selection operator used is a binary tournament similar to what NSGA-II uses. However, unlike the binary tournament in NSGA-II, the tournament is now decided by the strength (fitness) function:

In [ ]:
def binary_tournament_fitness(population):
    individual1 = random.choice(population)
    individual2 = random.choice(population)

    if individual1.fitness < individual2.fitness:
        return individual1
    elif individual1.fitness > individual2.fitness:
        return individual2
    else:
        return random.choice([individual1, individual2])

Selected individuals are subjected to crossover and mutation as usual:

In [ ]:
def generate_offspring_spea2(population):
    offspring_population = []
    while len(offspring_population) < population_size:
        parent1, parent2 = binary_tournament_fitness(population), binary_tournament_fitness(population)
        if random.random() < P_xover:
            offspring1, offspring2 = crossover(parent1, parent2)
        else:
            offspring1, offspring2 = parent1, parent2
        offspring1 = mutate(offspring1)
        offspring2 = mutate(offspring2)

        offspring_population.append(offspring1)
        offspring_population.append(offspring2)

    return offspring_population

Now we have all the components in place and can implement the overall algorithm:

In [ ]:
def spea2():
    initialise_plot()
    population = [get_random_individual() for _ in range(population_size)]
    archive = []

    for iteration in range(max_gen):
        plot(population+archive)

        calculate_strength(population, archive)

        # Copy all non-dominated individuals in population and archive to next archive
        next_archive = []
        for i in population + archive:
            if i.fitness < 1:
                next_archive.append(i)

        if len(next_archive) < population_size:
            pad_archive(next_archive, population, archive)

        if len(next_archive) > population_size:
            truncate(next_archive)


        # Mating selection:
        population = generate_offspring_spea2(next_archive)
        archive = next_archive

        # Termination: return non-dominated individuals in next_archive
        result = [p for p in next_archive if p.fitness < 1]

    plot(result)

    return result

In [ ]:
result = spea2()

In [ ]:
im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000, blit=True)
HTML(im_ani.to_jshtml())

As usual we add some datapoints for this algorithm.

In [ ]:
fronts_spea2 = run_times(spea2, 10)
results["SPEA2"] = [hypervolume(front, (0,total_costs)) for front in fronts_spea2]

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(results.values())
ax.set_xticklabels(results.keys())

## Two Archives

The Two Archives algorithm generalises the idea of using archives and uses one for convergence, and another one for diversity:

- If a new solution is not dominated by both archives and dominates at least one solution in either archive, it goes into the convergence archive. 
- If a new solution is not dominated by both archives but fails to dominate any solution in either archive, it goes into the diversity archive. 
- When the archives get full, the convergence archive is preserved while the diversity archive is pruned based on crowding distance. 

Praditwong, K., & Yao, X. (2006, November). A new multi-objective evolutionary optimisation algorithm: The two-archive algorithm. In 2006 International Conference on Computational Intelligence and Security (Vol. 1, pp. 286-291). IEEE.

The general workflow is as follows:
1. Initialise the population (as usual)
2. Initialise both archives as empty sets
3. Evaluate the initial population
4. Repeat until done:
5. Collect non-dominated individuals to the archives
6. Generate new population with parents from archives
7. Evaluate new population

During selection, one of the archives is probabilistically chosen. The probability is a pre-defined parameter that is a ratio to choose members from the convergence archive to the diversity archive. A member in the chosen archive is selected uniformly at random.

In [ ]:
def twoarchive_selection(convergence_archive, diversity_archive):
    if not convergence_archive:
        return random.choice(diversity_archive)
    if not diversity_archive:
        return random.choice(convergence_archive)
    if random.random() < 0.5:
        return random.choice(convergence_archive)
    else:
        return random.choice(diversity_archive)

The selection is the only difference in terms of reproduction compared to other evolutionary algorithms we have seen previously:

In [ ]:
def twoarchive_generate_offspring(convergence_archive, diversity_archive):
    offspring_population = []
    while len(offspring_population) < population_size:
        parent1 = twoarchive_selection(convergence_archive, diversity_archive)
        parent2 = twoarchive_selection(convergence_archive, diversity_archive)
        if random.random() < P_xover:
            offspring1, offspring2 = crossover(parent1, parent2)
        else:
            offspring1, offspring2 = parent1, parent2
        offspring1 = mutate(offspring1)
        offspring2 = mutate(offspring2)

        offspring_population.append(offspring1)
        offspring_population.append(offspring2)

    return offspring_population

We need to select all individuals from the resulting population that are not dominated by the archives. For this, we need some helper functions. First, we define a helper function that tells us whether a candidate solution is dominated by an archive:

A new individual is first compared with all members in the current archives. If it is dominated by a member of the archives, it is discarded, otherwise it becomes a new member of the archives.

The remainder of the archives are compared with the new member and two cases are possible:
- If the new member dominates a member of the archives then the dominated member is removed and the new member is received by the _convergence_ archive 
- If the new member does not dominate any members and is not dominated by any archive members then it becomes a member of the diversity archive, and the size of the diversity archive is increased. 

In [ ]:
def collect_nondominated(individual, ca, da):
    # TODO: During this stage, any duplicated member is deleted.

    if is_dominated(individual, ca) or is_dominated(individual, da):
        return

    dominated_ca = get_dominated(individual, ca)
    dominated_da = get_dominated(individual, da)
    if len(dominated_ca) + len(dominated_da) > 0:
        for i in dominated_ca:
            ca.remove(i)
        for i in dominated_da:
            da.remove(i)
        ca.append(individual)
    else:
        # No individuals are dominated
        da.append(individual)

If the total size of the archives overflows, then we need to remove individuals. The removal operator deletes only members in the diversity archive and has no impact on the convergence archive. 

To select which members to remove from the diversity archive, we calculate the shortest distance to the nearest member in the convergence archive for all members in the diversity archive. The member with the shortest distance among the diversity members is deleted until the total size equals the capacity. 

In [ ]:
archive_size = 40

def remove(ca, da):
    if len(ca) + len(da) > archive_size:
        for individual in da:
            individual.length = sys.maxsize
            for other in ca:
                d = get_distance(individual, other)
                if individual.length > d:
                    individual.length = d
        da.sort(key=lambda r: r.length)
        while len(ca) + len(da) > archive_size:
            del da[0]

Now we just need to put everything together. 

In [ ]:
def twoarchives():
    initialise_plot()
    population = [get_random_individual() for _ in range(population_size)]
    convergence_archive = []
    diversity_archive   = []

    for iteration in range(max_gen):
        plot(population+convergence_archive+diversity_archive)

        # Collect non-dominated individuals
        for i in get_nondominated(population):
            collect_nondominated(i, convergence_archive, diversity_archive)

        # Truncate archive sizes
        remove(convergence_archive, diversity_archive)

        next_generation = twoarchive_generate_offspring(convergence_archive, diversity_archive)
        population = next_generation

    result = get_nondominated(convergence_archive)
    plot(result)
    return result

In [ ]:
result = twoarchives()

In [ ]:
im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000, blit=True)
HTML(im_ani.to_jshtml())

In [ ]:
fronts_ta = run_times(twoarchives, 10)
results["TwoArchives"] = [hypervolume(front, (0,total_costs)) for front in fronts_ta]

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(results.values())
ax.set_xticklabels(results.keys())

## SMS-EMOA

We've been comparing the multi-objective algorithms in terms of the hypervolume, although they used domination and diversity measurements to guide the search. An alternative is to use the hypervolume _directly_ to drive the search. The hypervolume is often referred to as S metric, and the idea to optimise the hypervolume is implemented in the S metric selection evolutionary multi objective algorithm (SMS-EMOA):

Beume, N., Naujoks, B., & Emmerich, M. (2007). SMS-EMOA: Multiobjective selection based on dominated hypervolume. European Journal of Operational Research, 181(3), 1653-1669.

The overall principle of SMS-EMOA is the following:
1. Generate a random population of individuals
2. Repeat until done:
3. Generate an offspring by variation
4. Replace existing member of population if it improves the hypervolume

The _reduce_ function sorts the population using fast non-dominated sort (as introduced with NSGA-II), and then discards one individual from the worst front. The choice of which individual to discard is determined by the contribution the hypervolume: The individual contributing least to the hypervolume is the one discarded.

In [ ]:
def reduce(population):
    fronts = fast_non_dominated_sort(population)
    last_front = [population[index] for index in fronts[-1]]
    if len(last_front) > 1:
        hypervolumes = []
        for i in range(len(last_front)):
            front = last_front[:i] + last_front[i+1:]
            hypervolumes.append(hypervolume(front, (0,total_costs)))
        chosen = last_front[hypervolumes.index(max(hypervolumes))]
        population.remove(chosen)
    else:
        population.remove(last_front[0])

Since the calculation of the hypervolume can be computationally expensive,the algorithm is implemented as a steady state algorithm.

In [ ]:
def sms():
    initialise_plot()
    population = [get_random_individual() for _ in range(population_size)]

    for step in range(max_gen * population_size):
        if step % population_size == 0:
            plot(population)

        parent1, parent2 = random.choice(population), random.choice(population)
        if random.random() < P_xover:
            offspring1, offspring2 = crossover(parent1, parent2)
        else:
            offspring1, offspring2 = parent1, parent2
        
        population.append(mutate(random.choice([offspring1, offspring2])))
        reduce(population)

    result = get_nondominated(population)
    plot(result)
    return result 

In [ ]:
result = sms()

In [ ]:
im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000, blit=True)
HTML(im_ani.to_jshtml())

In [ ]:
fronts_sms = run_times(sms, 10)
results["SMS-EMOA"] = [hypervolume(front, (0,total_costs)) for front in fronts_sms]

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(results.values())
ax.set_xticklabels(results.keys())

This tells us which algorithm is best overall on our first NRP problem.

## Alternative Next Release Problem

In [ ]:
profit_map = {}        # customer id => weight
requirements_map = {}  # customer id => [ requirement_id * ]
cost_map = {}          # requirement id => cost
dependency_map = {}    # requirement id => [ requirement_id * ]
num_requirements = 620
parse_nrp("data/nrp/nrp2.txt")
total_costs = sum([ cost_map[i+1] for i in range(num_requirements)])
total_profit = sum(profit_map.values())

In [ ]:
results = {}

In [ ]:
fronts_random = run_times(random_moo, 10)
fronts_nsgaii = run_times(nsgaii, 10)
fronts_paes = run_times(paes, 10)
fronts_spea2 = run_times(spea2, 10)
fronts_ta = run_times(twoarchives, 10)
fronts_sms = run_times(sms, 10)

results["Random"]   = [hypervolume(front, (0,total_costs)) for front in fronts_random]
results["NSGA-II"]  = [hypervolume(front, (0,total_costs)) for front in fronts_nsgaii]
results["PAES"] = [hypervolume(front, (0,total_costs)) for front in fronts_paes]
results["SPEA2"] = [hypervolume(front, (0,total_costs)) for front in fronts_spea2]
results["TwoArchives"] = [hypervolume(front, (0,total_costs)) for front in fronts_ta]
results["SMS-EMOA"] = [hypervolume(front, (0,total_costs)) for front in fronts_sms]

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(results.values())
ax.set_xticklabels(results.keys())

## Comparison

There is a range of benchmark problems to compare multi-objective optimisation algorithms on. For example, we will consider the ZDT set of functions defined in the following paper:

Chase, N., Rademacher, M., Goodman, E., Averill, R., & Sidhu, R. (2009). A benchmark study of multi-objective optimization methods. BMK-3021, Rev, 6, 1-24.

The ZDT1 function has a convex Pareto-optimal front. The objective functions are

$f_1(x) = x_1$
$f_2(x) = g(x) [ 1 - \sqrt{x_1 / g(x)} ]$

where $g(x)$ is defined as:

$g(x) = 1 + 9 (\Sigma^n_{i=2} x_i) / (n-1)$.

Individuals are vectors of floating point numbers of size $n$:

In [ ]:
n = 30

In [ ]:
def function1(individual):
    return individual[0]

In [ ]:
def function2(individual):
    x0 = individual[0]

    sum = 0.0;
    for x in individual[1:]:
        sum += x
        
    g = 1.0 + 9.0 * sum / (len(individual) - 1)

    return g * (1 - sqrt(x0 / g))

Our representation currently only gives us bitvectors, so we need to adapt the search operators to produce vectors of floating point numbers.

In [ ]:
def get_random_individual():
    individual = L(random.random() for _ in range(n))
    evaluate(individual)
    return individual

To mutate a floating point number, we add some random noise using a Gaussian distribution.

In [ ]:
def mutate(solution):
    P_mutate = 1/len(solution)
    mutated = L(solution[:])
    for position in range(len(solution)):
        if random.random() < P_mutate:
            mutated[position] = max(0, min(1, mutated[position] + random.gauss(0, 0.05)))
    evaluate(mutated)
    return mutated

Both our objectives are minimisation problems, so we need to update the dominance relation accordingly.

In [ ]:
def dominates(solution1, solution2):
    if solution1.fitness1 > solution2.fitness1 or solution1.fitness2 > solution2.fitness2:
        return False

    if solution1.fitness1 < solution2.fitness1 or solution1.fitness2 < solution2.fitness2:
        return True

    return False

We also need to remove the normalisation based on total profit and costs.

In [ ]:
max_fitness1 = function1([1 for _ in range(n)])
max_fitness2 = function2([1 for _ in range(n)])

In [ ]:
def hypervolume(front, r):
    front.sort(key=lambda i: i.fitness1)

    hv = (abs(r[0] - front[0].fitness1) / max_fitness1) * (abs(r[1] - front[0].fitness2) / max_fitness2)
    
    for i in range(1, len(front)):
        hv += (abs(front[i-1].fitness1 - front[i].fitness1) / max_fitness1) * (abs(r[1] - front[i].fitness2) / max_fitness2)
    
    return hv

Finally, we also need to fix the axis labels on our animation plots.

In [ ]:
ims = []  # global variable to store images of the animation
    
def initialise_plot():
    global ims
    global fig
    global ax
    
    ims = []

    %matplotlib agg
    fig, ax = plt.subplots()
    plt.xlabel('Function 1', fontsize=15)
    plt.ylabel('Function 2', fontsize=15)
    ims = []
    %matplotlib inline

In [ ]:
random_moo()

In [ ]:
im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000, blit=True)
HTML(im_ani.to_jshtml())

In [ ]:
result = nsgaii()

In [ ]:
im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000, blit=True)
HTML(im_ani.to_jshtml())

In [ ]:
results = {}

fronts_random = run_times(random_moo, 10)
fronts_nsgaii = run_times(nsgaii, 10)
fronts_paes = run_times(paes, 10)
fronts_spea2 = run_times(spea2, 10)
fronts_ta = run_times(twoarchives, 10)
fronts_sms = run_times(sms, 10)

results["Random"]   = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_random]
results["NSGA-II"]  = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_nsgaii]
results["PAES"] = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_paes]
results["SPEA2"] = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_spea2]
results["TwoArchives"] = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_ta]
results["SMS-EMOA"] = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_sms]

fig, ax = plt.subplots()
ax.boxplot(results.values())
ax.set_xticklabels(results.keys());

For ZDT4, $f_1$ remains the same, but $f_2 = g(x)[1 - \sqrt{x_1/g(x)}]$, where $g(x) = 1 + 10(n - 1) + \Sigma^n_{i=2} [x_i^2 - 10 cos (4\pi{}x_i)]$. This results in a highly multi-modal fitness landscape.

In [ ]:
from math import cos, pi

def function2(individual):
    x0 = individual[0]

    g = 0
    for x in individual[1:]:
        g += pow(x, 2) - 10.0 * cos(4.0 * pi * x / 180.0)
        
    g += 1.0 + 10.0 * (len(individual) - 1)

    return g * (1.0 - sqrt(x0 / g))

In [ ]:
max_fitness1 = function1([1 for _ in range(n)])
max_fitness2 = function2([1 for _ in range(n)])

In [ ]:
results = {}

fronts_random = run_times(random_moo, 10)
fronts_nsgaii = run_times(nsgaii, 10)
fronts_paes = run_times(paes, 10)
fronts_spea2 = run_times(spea2, 10)
fronts_ta = run_times(twoarchives, 10)
fronts_sms = run_times(sms, 10)

results["Random"]   = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_random]
results["NSGA-II"]  = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_nsgaii]
results["PAES"] = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_paes]
results["SPEA2"] = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_spea2]
results["TwoArchives"] = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_ta]
results["SMS-EMOA"] = [hypervolume(front, (max_fitness1,max_fitness2)) for front in fronts_sms]

fig, ax = plt.subplots()
ax.boxplot(results.values())
ax.set_xticklabels(results.keys())